In [3]:
import torch
import random

In [4]:
batch_size = 3

In [5]:
edge_index = torch.LongTensor([[11, 12, 13, 14, 15, 16, 17],
                               [102, 212, 132, 232, 402, 113, 122]])
num_nodes = edge_index.size(1)

In [8]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def sample_neg( edge_index):
    edge_index = edge_index.to(device)
    num_nodes = edge_index.size(1)
    user_indices, pos_item_indices = edge_index

    neg_item_indices = torch.empty(0, dtype=torch.int64, device=device)  # 指定数据类型和设备
    for i in range(num_nodes):
        random_index = torch.randint(0, edge_index.size(1), (1,), device=device)
        while random_index.item() == edge_index[1, i].item():
            random_index = torch.randint(0, edge_index.size(1), (1,), device=device)
        random_element = edge_index[1, random_index]
        neg_item_indices = torch.cat((neg_item_indices, random_element), dim=0)

    edge_index_new = torch.stack((user_indices, pos_item_indices, neg_item_indices), dim=0)
    indices = random.choices([i for i in range(edge_index_new[0].shape[0])], k=num_nodes)
    print("sample_edges: ", edge_index_new)
    batch = edge_index_new[:, indices]
    user_indices, pos_item_indices, neg_item_indices = batch[0], batch[1], batch[2]
    print(batch)

    return batch
# 示例使用
batch = sample_neg( edge_index)
print(batch)

sample_edges:  tensor([[ 11,  12,  13,  14,  15,  16,  17],
        [102, 212, 132, 232, 402, 113, 122],
        [122, 102, 402, 402, 212, 132, 113]], device='cuda:0')
tensor([[ 14,  15,  17,  17,  13,  15,  12],
        [232, 402, 122, 122, 132, 402, 212],
        [402, 212, 113, 113, 402, 212, 102]], device='cuda:0')
tensor([[ 14,  15,  17,  17,  13,  15,  12],
        [232, 402, 122, 122, 132, 402, 212],
        [402, 212, 113, 113, 402, 212, 102]], device='cuda:0')


In [24]:
print(user_indices, pos_item_indices, neg_item_indices)

tensor([16, 16, 17], device='cuda:0') tensor([113, 113, 122], device='cuda:0') tensor([132, 132, 232], device='cuda:0')


In [24]:

import pandas as pd
import csv
movie_path = '../data/ml-1m/movies.dat'
rating_path = '../data/ml-1m/ratings.dat'

with open(movie_path, 'r',encoding='utf-8') as file:
    content = file.read()


def convert_dat_to_csv(dat_data, csv_file_path):
    """
    Converts dat format data to CSV format.

    Args:
        dat_data (str): The content of the dat file as a string.
        csv_file_path (str): The path where the CSV file will be saved.
    """
    # 分割数据为行
    lines = dat_data.strip().split('\n')

    # 创建CSV文件并写入数据
    with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file, delimiter='::')

        # 写入标题行
        writer.writerow(['movieId', 'title', 'genres'])

        # 遍历每一行数据
        for line in lines:
            if line.strip():  # 确保不处理空行
                # 分割每行为movieId, title, genres
                parts = line.split('::')
                if len(parts) == 3:
                    movieId, title, genres = parts
                    writer.writerow([movieId, title, genres])

convert_dat_to_csv(content, 'movies.csv')
#print(content)
